## Import iris data set

In [1]:
from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

SVC()

In [3]:
# iris_classifier.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    """
    A minimum prediction service exposing as Scikit-learn model
    """
    
    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        An inference API named 'predict' with Dataframe input adapter, which
        codified how HTTP requests or CSV files are converted to a pandas 
        Dataframe object as the inference API function input
        """
        return self.artifacts.model.predict(df)

## Save prediction service for distribution

In [6]:
#import the IrisClassifier class defined above
from iris_classifier import IrisClassifier

# Create a iris classifier service instance
iris_classifier_service = IrisClassifier()

# Pack the newly trained model artifact
iris_classifier_service.pack('model', clf)

# Save the prediction service to disk for model serving
saved_path = iris_classifier_service.save()

[2021-04-19 22:48:38,208] WARNING - pip package requirement pandas already exist
[2021-04-19 22:48:38,210] WARNING - pip package requirement scikit-learn already exist
[2021-04-19 22:48:39,776] INFO - BentoService bundle 'IrisClassifier:20210419224838_A8FA22' saved to: C:\Users\cwryu/bentoml/repository/IrisClassifier\20210419224838_A8FA22


### Model Serving via Rest API

In [7]:
import requests
response = requests.post("http://127.0.0.1:5000/predict", json=[[5.1, 3.5, 1.4, 0.2]])
print(response.text)

[0]
